# Multi-Path Effect with Raytracing

This is an example of using `RadarSimPy` to simulate an FMCW radar with a corner reflector model above the ground surface. Ray-Tracing/Shoot-and-Bounce-Rays method is used in `RadarSimPy`. The ray-tracing engine `RadarSimC`, which is built with C++, is integrated in RadarSimPy.

>`RadarSimPy` is a radar simulation package built with python. **<a href="https://zpeng.me/#contact" target="_blank">Contact me</a> if you are interested in this module.**

This notebook is available on my <a href="https://github.com/rookiepeng/radar-simulation-notebooks" target="_blank">GitHub</a>.

---

## Setup FMCW radar

### Transmitter

The following table lists the basic parameters of the radar transmitter.

 | Parameter                       | Variable in `RadarSimPy` | Value      |
 |---------------------------------|--------------------------|------------|
 | Center frequency ($f_c$)        | fc                       | 77 GHz     |
 | Bandwidth ($B$)                 | bandwidth                | 100 MHz    |
 | Transmitted power ($P_t$)       | tx_power                 | 10 dBm     |
 | Rising or falling chirp         | slop_type                | rising     |
 | Chirp repetition period ($CRP$) | repetition_period        | 100 us     |
 | Chirp length ($T$)              | pulse_length             | 80 us      |
 | Number of chirps                | pulses                   | 256        |
 | Chirp start delay               | delay                    | 0 s        |
 
### Receiver
 The parameters of the receiver are listed in the table below.

 | Parameter                        | Variable in `RadarSimPy` | Value        |
 |----------------------------------|--------------------------|--------------|
 | Sampling rate ($f_s$)            | fs                       | 2 Msps       |
 | Noise figure ($NF$)              | noise_figure             | 12 dB        |
 | RF gain/loss ($G_{rf}$)          | rf_gain                  | 20 dB        |
 | Load resistor ($R_L$)            | load_resistor            | 500 $\Omega$ |
 | Baseband voltage gain ($G_{BB}$) | baseband_gain            | 30 dB        |


In [1]:
import numpy as np
from radarsimpy import Radar, Transmitter, Receiver

angle = np.arange(-90, 91, 1)
pattern = 20 * np.log10(np.cos(angle / 180 * np.pi) + 0.01)

tx_channel = dict(
    location=(0, 0, 0),
    azimuth_angle=angle,
    azimuth_pattern=pattern,
    elevation_angle=angle,
    elevation_pattern=pattern,
)

tx = Transmitter(fc=77e9,
                 pulse_length=80e-6,
                 bandwidth=100e6,
                 tx_power=15,
                 slop_type='rising',
                 repetition_period=100e-6,
                 pulses=256,
                 channels=[tx_channel])

rx_channel = dict(
    location=(0, 0, 0),
    azimuth_angle=angle,
    azimuth_pattern=pattern,
    elevation_angle=angle,
    elevation_pattern=pattern,
)

rx = Receiver(fs=2e6,
              noise_figure=8,
              rf_gain=20,
              load_resistor=500,
              baseband_gain=30,
              channels=[rx_channel])

aperture = np.zeros((2, 3, 3))

aperture[0, 0, 0] = -0.25
aperture[0, 0, 1] = 0.0
aperture[0, 0, 2] = -0.25
aperture[0, 1, 0] = -0.25
aperture[0, 1, 1] = 0.0
aperture[0, 1, 2] = 0.25
aperture[0, 2, 0] = 0.25
aperture[0, 2, 1] = 0.0
aperture[0, 2, 2] = -0.25
aperture[1, 0, 0] = 0.25
aperture[1, 0, 1] = 0.0
aperture[1, 0, 2] = -0.25
aperture[1, 1, 0] = 0.25
aperture[1, 1, 1] = 0.0
aperture[1, 1, 2] = 0.25
aperture[1, 2, 0] = -0.25
aperture[1, 2, 1] = 0.0
aperture[1, 2, 2] = 0.25

radar = Radar(transmitter=tx, receiver=rx, type='FMCW', aperture=aperture)

## Case 1: A corner reflector with ground surface

In [2]:
from radarsimpy import scene
import radarsimpy.simulator as sim
from scipy import signal
import radarsimpy.processing as proc

t_range = np.arange(30, 60, 0.25)

amp_multi = np.zeros_like(t_range)

for r_idx in range(0, len(t_range)):
    target_1 = {
        'model': '../models/cr.stl',
        'location': (0, t_range[r_idx], 0),
        'speed': (0, 0, 0)
    }

    target_2 = {
        'model': '../models/surface_400x400.stl',
        'location': (0, 0, -1),
        'speed': (0, 0, 0),
        'is_ground': True
    }

    targets = [target_1, target_2]

    rays = scene(radar, targets, density=0.5)
    data = sim.run_simulator(radar, rays, is_raytracing=True, correction=-42)
    baseband = data['baseband']
    range_window = signal.chebwin(radar.samples_per_pulse, at=60)
    range_profile = proc.cal_range_profile(radar, baseband, range_window)

    doppler_window = signal.chebwin(radar.transmitter.pulses, at=60)
    range_doppler = proc.cal_range_doppler(
        radar, range_profile, doppler_window, fft_shift=False)

    amp_multi[r_idx] = np.max(np.abs(range_doppler[0, :, :]))

## Case 2: A corner reflector without ground surface

In [3]:
amp_single = np.zeros_like(t_range)

for r_idx in range(0, len(t_range)):
    target_1 = {
        'model': '../models/cr.stl',
        'location': (0, t_range[r_idx], 0),
        'speed': (0, 0, 0)
    }

    targets = [target_1]

    rays = scene(radar, targets, density=0.5)
    data = sim.run_simulator(radar, rays, is_raytracing=True, correction=-42)
    baseband = data['baseband']
    range_window = signal.chebwin(radar.samples_per_pulse, at=60)
    range_profile = proc.cal_range_profile(radar, baseband, range_window)

    doppler_window = signal.chebwin(radar.transmitter.pulses, at=60)
    range_doppler = proc.cal_range_doppler(
        radar, range_profile, doppler_window, fft_shift=False)

    amp_single[r_idx] = np.max(np.abs(range_doppler[0, :, :]))

## Results

Duo to the multi-path:
- Path 1: Tx -> Corner reflecor -> Rx
- Path 2: Tx -> Ground -> Corner reflecor -> Ground -> Rx

A fluctuation can be observed from the received target amplitude versus the target range. 

In [6]:
import plotly.graph_objs as go
from plotly.offline import iplot

p_multi = go.Scatter(x=t_range, y=20*np.log10(amp_multi),
                     name='CR with Ground')
p_single = go.Scatter(x=t_range, y=20*np.log10(amp_single),
                      name='CR without Ground')

layout = go.Layout(
    title='Multi-Path Effect',
    yaxis=dict(title='Amplitude (dB)'),
    xaxis=dict(title='Range (m)'),
)
fig = go.Figure(data=[p_multi, p_single], layout=layout)
iplot(fig)